In [20]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(rc={'figure.figsize':(11.7,8.27)})
sns.set_style("white")
sys.path.append('..')
import datetime as DT
import os
import rbf_functions

In [21]:
rbfs = [rbf_functions.original_rbf,
        rbf_functions.squared_exponential_rbf,
        rbf_functions.inverse_quadratic_rbf,
        rbf_functions.inverse_multiquadric_rbf,
        rbf_functions.exponential_rbf,
        rbf_functions.matern32_rbf,
        rbf_functions.matern52_rbf,
       ]

# Load model output for each activation function

In [22]:
nfearchive = {}
for n in rbfs:
    nfearchive[n.__name__] = {}
for entry in rbfs:
    name = entry.__name__
    output_dir = f"../output/{name}/"
    for i in os.listdir(output_dir):
        if i.endswith("_hypervolume.csv"):
            archives_by_nfe = pd.read_csv(output_dir + i)
            nfes = archives_by_nfe["Unnamed: 0"].values
            u_nfes = np.unique(nfes)
            selected_nfe = u_nfes[0::5]
            selected_nfe = np.append(selected_nfe, u_nfes[-1::])
            nfearchive[name][int(i.split("_")[0])] = archives_by_nfe.loc[archives_by_nfe['Unnamed: 0'].isin(selected_nfe)]

# Transform the dataframe back into platypus solution instances, to have a collection of solutions for each generation

In [23]:
from platypus import Solution, Problem, GenerationalDistance

n_rbfs = 4
n_objs = 6
n_vars = n_rbfs * 8 

problem = Problem(n_vars, n_objs)

# matters for hypervolume
problem.directions[0] = Problem.MAXIMIZE  # hydropower
problem.directions[1] = Problem.MAXIMIZE  # atomic power plant
problem.directions[2] = Problem.MAXIMIZE  # baltimore
problem.directions[3] = Problem.MAXIMIZE  # chester
problem.directions[4] = Problem.MINIMIZE  # environment
problem.directions[5] = Problem.MAXIMIZE  # recreation

archives = {}
seeds = np.arange(10, 101, 10).tolist()
for n in rbfs:
    archives[n.__name__] = {}
    for i in seeds:
        archives[n.__name__][i] = {}
for entry in rbfs:
    name = entry.__name__
    for s in nfearchive[name]: #nfearchive[name]:
        for nfe, generation in nfearchive[name][s].groupby("Unnamed: 0"):
            # we slice from 2, getting rid of the first two columns 
            # which contain the NFE and ID of each solution
            archive = []
            for i, row in generation.iloc[:, 2::].iterrows():
                solution = Solution(problem)
                solution.objectives = row
                archive.append(solution)
            archives[name][s][nfe] = archive
        

# Load refsets for each rbf, turn into solutions sets

In [25]:
# load refsets for each rbf
ref_dir = "./refsets/"
ref_sets = {}
for n in rbfs:
    name = n.__name__
    ref_sets[name] = {}
    data = pd.read_csv(f'{ref_dir}{name}_refset.csv')  
    ref_set = []
    for i, row in data.iterrows():
        solution = Solution(problem)
        solution.objectives = row
        ref_set.append(solution)
    ref_sets[name] = ref_set

# Or, load global refset for all rbfs, turn into solution set

In [27]:
# load refset for all rbfs
ref_dir = "./refsets"
ref_sets = {}
data = pd.read_csv(f'{ref_dir}/global_refset.csv')  
ref_set = []
for i, row in data.iterrows():
    solution = Solution(problem)
    solution.objectives = row
    ref_set.append(solution)

# Calculate generational distance

In [ ]:
# gd = GenerationalDistance(reference_set=ref_set)
# nfes = []
# gds = []
# for nfe, archive in archives.items():
#     nfes.append(nfe)
#     gds.append(gd.calculate(archive))
    
tempnfe = {}
tempgd = {}
nfe_sets = {}
gd_sets = {}
for rbf in archives:
    nfe_sets[rbf] = {}
    gd_sets[rbf] = {}
#     gd = GenerationalDistance(reference_set=ref_sets[rbf])
    gd = GenerationalDistance(reference_set=ref_set) #global refset
    print(f"started {rbf} at {DT.datetime.now().strftime('%H:%M:%S')}")
    for seed in archives[rbf]:
        nfe_sets[rbf][seed] = {}
        gd_sets[rbf][seed] = {}
        s_archives = archives[rbf][seed]
        nfes = []
        gds = []
        for nfe, archive in s_archives.items():
            nfes.append(nfe)
            gds.append(gd.calculate(archive))
        nfe_sets[rbf][seed] = nfes
        gd_sets[rbf][seed] = gds
        tempnfe[seed] = nfes
        tempgd[seed] = gds
        dfgd = pd.DataFrame.from_dict(tempgd, orient='index')
        dfnfe = pd.DataFrame.from_dict(tempnfe, orient='index')
        dfgd = dfgd.T
        dfnfe = dfnfe.T
#         dfgd.to_csv(f"gd/gd/{rbf}_gd.csv", index=False)
#         dfnfe.to_csv(f"gd/gd/{rbf}_gd_nfe.csv", index=False)
        dfgd.to_csv(f"gd/gd_global/{rbf}_gd_all.csv", index=False) #global refset
        dfnfe.to_csv(f"gd/gd_global/{rbf}_gd_nfe_all.csv", index=False) #global refset
        print(f"finished seed: {seed} at {DT.datetime.now().strftime('%H:%M:%S')}")

started original_rbf at 21:11:28
finished seed: 10 at 21:14:48
finished seed: 20 at 21:18:04
finished seed: 30 at 21:21:34
finished seed: 40 at 21:25:11
finished seed: 50 at 21:28:53
finished seed: 60 at 21:31:56
finished seed: 70 at 21:35:05
finished seed: 80 at 21:38:22
finished seed: 90 at 22:11:45
finished seed: 100 at 02:51:23
started squared_exponential_rbf at 02:51:23
finished seed: 10 at 02:55:10


# Plot results

In [ ]:
sns.set_style("white")
for entry in rbfs:
    name = entry.__name__
    gd = pd.DataFrame.from_dict(gd_sets[name], orient='index').T
    nfe = pd.DataFrame.from_dict(nfe_sets[name], orient='index').T
    
    fig, ax = plt.subplots()
    ax.plot(nfe, gd)
    ax.set_xlabel('nfe')
    ax.set_ylabel('Generational distance')
    ax.set_title(name)
    sns.despine()

    plt.show()

# Local reference set

In [ ]:
for entry, c in zip(rbfs, sns.color_palette()):
    rbf = entry.__name__
    df_gd = pd.read_csv(f"gd/{rbf}_gd.csv")
    df_gd_nfe = pd.read_csv(f"gd/{rbf}_gd_nfe.csv")
    fig, ax = plt.subplots(sharex=True)

    ax.plot(df_gd_nfe, df_gd, color=c)

    ax.set_xlabel('nfe')
    ax.set_ylabel('epsilon indicator')
    ax.set_title(rbf)
    sns.despine()
    plt.savefig(f'figs/{rbf}/{rbf}_gd.png')
plt.show()

# Global reference set

In [ ]:
for entry, c in zip(rbfs, sns.color_palette()):
    rbf = entry.__name__
    df_gd = pd.read_csv(f"gd_global/{rbf}_gd_all.csv")
    df_gd_nfe = pd.read_csv(f"gd_global/{rbf}_gd_nfe_all.csv")
    fig, ax = plt.subplots(sharex=True)

    ax.plot(df_gd_nfe, df_gd, color=c)

    ax.set_xlabel('nfe')
    ax.set_ylabel('epsilon indicator')
    ax.set_title(rbf)
    sns.despine()
#     plt.savefig(f'figs/{rbf}/{rbf}_gd_global.png')
plt.show()